In [1]:
import folium
from folium import GeoJson, GeoJsonTooltip, LayerControl
from branca.colormap import LinearColormap
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
from IPython.display import IFrame
import csv
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from folium.raster_layers import WmsTileLayer


In [2]:


def load_counts(csv_path):
    rows = []
    with open(csv_path, "r", newline="") as f:
        rdr = csv.DictReader(f)
        for r in rdr:
            gid = r["grid_id"]
            cnt = float(r["count"])
            try:
                lon, lat = map(float, gid.split("_")[-2:])
            except Exception:
                continue
            rows.append((lon, lat, cnt))
    return rows


In [3]:
rows = load_counts("G:\\Downloads\\251202analysis_res\\20251202_test.csv")


In [4]:

# 你的网格多边形（WGS84）
dx=0.1
dy=0.1
polys, vals = [], []

for lon, lat, cnt in rows:
    # 右下角是 (lon, lat)；构建矩形：
    # 左下( lon-dx, lat-dy )、右下( lon, lat-dy )、右上( lon, lat )、左上( lon-dx, lat )
    # 根据你真实网格定义调整（这里仅示例）
    poly = Polygon([
        (lon, lat),
        (lon + dx,    lat),
        (lon + dx,    lat + dy),
        (lon, lat + dy),
    ])
    polys.append(poly)
    vals.append(cnt)

gdf = gpd.GeoDataFrame({"count": vals}, geometry=polys, crs="EPSG:4326")


minx, miny, maxx, maxy = gdf.total_bounds
center = [(miny + maxy) / 2, (minx + maxx) / 2]  # [lat, lon]

In [5]:
# 建图时不要内置默认底图
m = folium.Map(location=center, zoom_start=11, tiles=None,
               control_scale=True, prefer_canvas=True)

# --- 帮助函数 ---
def add_base(name, url, attr, default=False, **kw):
    """互斥的基底图：一次只显示一个；default 设 True 的那个会默认显示"""
    folium.TileLayer(
        tiles=url, name=name, attr=attr,
        overlay=False, control=True, show=default,  # 关键：base 互斥，且只让一个 show=True
        no_wrap=True, max_zoom=22, max_native_zoom=19, **kw
    ).add_to(m)

def add_overlay(name, url, attr, show=True, **kw):
    """可叠加图层：可多选多显；show=True 表示默认就显示"""
    folium.TileLayer(
        tiles=url, name=name, attr=attr,
        overlay=True, control=True, show=show,      # 关键：叠加层允许多显
        no_wrap=True, max_zoom=22, **kw
    ).add_to(m)

# --- 底图（互斥，只让一个默认显示） ---
add_base("CartoDB Positron（浅色）",
         "https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png",
         "© OpenStreetMap © CARTO", default=True)  # 默认显示这张
add_base("OSM 标准",
         "https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
         "© OpenStreetMap contributors", default=False)
add_base("CartoDB Dark Matter（深色）",
         "https://{s}.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}{r}.png",
         "© OpenStreetMap © CARTO", default=False)
add_base("Esri 卫星（World Imagery）",
         "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
         "Tiles © Esri — Source: Esri, Maxar, Earthstar Geographics, and the GIS User Community",
         default=False)

# --- 叠加层（可同时显示，默认就开） ---
# 1) 标签（常与卫星底图一起用）
add_overlay("标签（叠加层）",
            "https://{s}.basemaps.cartocdn.com/light_only_labels/{z}/{x}/{y}.png",
            "© OpenStreetMap © CARTO",
            show=False, opacity=0.9, zIndex=650)

# # 2) WorldCover（WMS 示例；想默认显示就 show=True，不想默认显示就改 False）
from folium.raster_layers import WmsTileLayer
WmsTileLayer(
    url="https://services.terrascope.be/wms/v2",
    layers="WORLDCOVER_2021_MAP",
    name="WorldCover 2021 (WMS)",
    fmt="image/png",
    transparent=True,
    overlay=True, control=True, show=False,  # ← 默认显示
    attr="© ESA WorldCover 2021, © Terrascope/VITO",
    no_wrap=True, opacity=0.9
).add_to(m)


In [6]:


# ----------------------------
# 3) 颜色映射（用 YlOrBr，和你 Matplotlib 保持一致）
vmin, vmax = float(np.nanmin(vals)), float(np.nanmax(vals))
# 采样 10 个颜色阶，LinearColormap 需要颜色列表
colors_ylorbr = [
    "#ffffe5", "#fff7bc", "#fee391", "#fec44f", "#fe9929",
    "#ec7014", "#cc4c02", "#993404", "#662506", "#3d1f00"
]
cmap = LinearColormap(colors=colors_ylorbr, vmin=vmin, vmax=vmax)
cmap.caption = "Count (YlOrBr)"  # 图例标题
cmap.add_to(m)

# ----------------------------
# 4) 以 GeoJSON 叠加多边形（按 count 上色，附 tooltip）
geojson = gdf.__geo_interface__  # 也可以用 gdf.to_json()

def style_function(feature):
    c = feature["properties"]["count"]
    return {
        "fillColor": cmap(c),
        "color": "#333333",      # 边线颜色
        "weight": 0.5,           # 边线宽度
        "fillOpacity": 0.65
    }

gj = GeoJson(
    data=geojson,
    name="数据集Grid",
    style_function=style_function,
    tooltip=GeoJsonTooltip(
        fields=["count"],
        aliases=["count:"],
        localize=True,
        sticky=True
    )
)
gj.add_to(m)


m.fit_bounds([[miny, minx], [maxy, maxx]])

folium.LayerControl(collapsed=False).add_to(m)

out_html = "20251202_grid_heatmap.html"
m.save(out_html)